In [1]:
import os
import smtplib
from datetime import datetime, timedelta, date, timezone
import calendar
from smtplib import SMTPException
from clickhouse_driver import Client
from clickhouse_driver.errors import ServerException
from hdbcli import dbapi
import snowflake.connector as snowflake
from decimal import Decimal

#add to pipeline
import pandas as pd
from pandas import merge, read_excel, read_csv
from numpy import sum as npsum
from numpy import ndarray
import requests
from requests.auth import HTTPBasicAuth
from pprint import pprint
import json, xmltodict
from lxml.etree import parse
import itertools

#add as SFSF pipeline
from re import match, search
from numpy import nan, isnan
#add as SFSF pipeline
from pandas import DataFrame, json_normalize, to_datetime
import requests
from requests.auth import HTTPBasicAuth
import json
from re import match
import sys


# add as ar exe program
import glob

# !conda install -y pyautogui --channel conda-forge
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [2]:
df_master = read_excel('Plano店舗マスタ.xlsx', index=0, sheet_name=0, engine='openpyxl')
df_master

,Unnamed: 0,企業コード,企業名,店番(HouseCode),店名,DCNO,JScaleCode,CokeOneCode,チャネル,県名,ボトラー,CCBJIエリア,CCBJIサブエリア,法人名称,週次レポート対象,POS有,新商品レポート対象,管理ボトラー,CCBJI_L3企業コード,CCBJI_L3企業名,E予備6,E予備7,E予備8,E予備9,E予備10,E予備11,E予備12,E予備13,E予備14,E予備15,E予備16,E予備17,E予備18,E予備19,E予備20,カスタマー属性1,カスタマー属性2,カスタマー属性3,カスタマー属性4,カスタマー属性5,カスタマー属性6,カスタマー属性7,カスタマー属性8,カスタマー属性9,カスタマー属性10,カスタマー属性11,カスタマー属性12,カスタマー属性13,カスタマー属性14,カスタマー属性15,カスタマー属性16,カスタマー属性17,カスタマー属性18,カスタマー属性19,カスタマー属性20,Unnamed: 55,Unnamed: 56
0,NaN,1172,ＭＶ北陸,25330,ＭＶ上飯野,エリア外,NaN,エリア外,SM,16_富山,北陸,北陸,北陸,ＭＶ北陸,NaN,●,NaN,CCBJI,エリア外,エリア外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
1,NaN,1172,ＭＶ北陸,25360,ＭＶ上市,エリア外,NaN,エリア外,SM,16_富山,北陸,北陸,北陸,ＭＶ北陸,NaN,●,NaN,CCBJI,エリア外,エリア外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
2,NaN,1172,ＭＶ北陸,25370,ＭＶ氷見,エリア外,NaN,エリア外,SM,16_富山,北陸,北陸,北陸,ＭＶ北陸,NaN,●,NaN,CCBJI,エリア外,エリア外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
3,NaN,1172,ＭＶ北陸,25380,ＭＶ羽咋,エリア外,NaN,エリア外,SM,17_石川,北陸,北陸,北陸,ＭＶ北陸,NaN,●,NaN,CCBJI,エリア外,エリア外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
4,NaN,1172,ＭＶ北陸,25770,ＭＶ金沢駅西本町,エリア外,NaN,エリア外,SM,17_石川,北陸,北陸,北陸,ＭＶ北陸,NaN,●,NaN,CCBJI,エリア外,エリア外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32368,NaN,52320A,サンプラザ・大阪全店計,-,全店計,-,NaN,登録なし,SM,27_大阪,CCBJI,近畿四国,近畿,サンプラザ・大阪,NaN,●,NaN,NaN,対象外,対象外,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
32369,NaN,3400127,コープこうべ・KC,-,全店計,NaN,NaN,登録なし,宅配,28_兵庫,CCBJI,近畿四国,近畿,コープこうべ・KC,NaN,●,NaN,NaN,3400127,ｺｰﾌﾟこうべ･KC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-28,洗替
32370,NaN,3156019A,ユタカドラッグ全店計,-,全店計,NaN,NaN,登録なし,DRUG,21_岐阜,CCBJI,中部日本,中部日本,NaN,NaN,NaN,NaN,CCBJI,3156019,ユタカドラッグ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,執行さん依頼
32371,NaN,3111014,イトーヨーカドー,-,全店計,NaN,NaN,NaN,SM,13_東京,CCBJI,東京,東京,NaN,NaN,NaN,NaN,CCBJI,3111014,イトーヨーカドー,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-22,米内山さん依頼


In [38]:
df = read_csv('C:/Users/tsubasa.ohmura/Downloads/Output/Output.csv')

MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64

In [19]:
df.iloc[0:, 20:55].any()

E予備6         False
E予備7         False
E予備8         False
E予備9         False
E予備10        False
E予備11        False
E予備12        False
E予備13        False
E予備14        False
E予備15        False
E予備16        False
E予備17        False
E予備18        False
E予備19        False
E予備20        False
カスタマー属性1      True
カスタマー属性2      True
カスタマー属性3      True
カスタマー属性4      True
カスタマー属性5      True
カスタマー属性6      True
カスタマー属性7      True
カスタマー属性8      True
カスタマー属性9      True
カスタマー属性10     True
カスタマー属性11     True
カスタマー属性12    False
カスタマー属性13    False
カスタマー属性14    False
カスタマー属性15    False
カスタマー属性16    False
カスタマー属性17    False
カスタマー属性18    False
カスタマー属性19    False
カスタマー属性20    False
dtype: bool

In [165]:
zjcust = read_excel('C:/Users/tsubasa.ohmura/Desktop/Storage/Project/Commercial BI Project/ClickHouse/ZJCUSTカラム定義.xlsx', engine='openpyxl')

In [166]:
def replace_space_to_underbar(dataframe, column_name):
    df = dataframe
    for i in range(len(df)):
        target = df[column_name][i]
        for l in range(len(target)):
            if target[l] == ' ':
                target = target[:l] + '_' + target[l+1:]
            df[column_name][i] = target
    return df
            
def replace_large_to_small(dataframe, column_name):
    df = dataframe
    for i in range(len(df)):
        target = str(df[column_name][i])
        target = target.lower()
        df[column_name][i] = target
    return df

In [167]:
zjcust = replace_space_to_underbar(zjcust, 'CH内のカラム名')
zjcust = replace_large_to_small(zjcust, 'CH内のカラム名')

In [168]:
zjcust

,カラムの意味,CH内のカラム名,HANA内のカラム名
0,納品場所コード,delivery_location_name,ZJCUST
1,納品場所,customer_name,ZJCUST___T
2,ロケーションコード,location_code,ZJLOCACD
3,ロケーション,location_name,ZJLOCACD___T
4,ディーラーコード,dealer_code,ZJDLRC
5,ディーラー,dealer_name,ZJDLRC___T
6,会社コード,company_code,ZJCUST__0COMP_CODE
7,会社,company_name,ZJCUST__0COMP_CODE___T
8,CCBJIチャネルLv.1コード,ccbji_channel_lv.1_code,ZJBJICLV1
9,CCBJIチャネルLv.1,ccbji_channel_lv.1_name,ZJBJICLV1___T


In [176]:
zjcust.to_csv('C:/Users/tsubasa.ohmura/Desktop/Storage/Project/Commercial BI Project/ClickHouse/zjcust_sql.csv', index=False, encoding='utf-8')

In [170]:
zjcust['sql'] = zjcust['HANA内のカラム名'] + ' as ' + zjcust['CH内のカラム名'] + ','
zjcust

,カラムの意味,CH内のカラム名,HANA内のカラム名,sql
0,納品場所コード,delivery_location_name,ZJCUST,"ZJCUST as delivery_location_name,"
1,納品場所,customer_name,ZJCUST___T,"ZJCUST___T as customer_name,"
2,ロケーションコード,location_code,ZJLOCACD,"ZJLOCACD as location_code,"
3,ロケーション,location_name,ZJLOCACD___T,"ZJLOCACD___T as location_name,"
4,ディーラーコード,dealer_code,ZJDLRC,"ZJDLRC as dealer_code,"
5,ディーラー,dealer_name,ZJDLRC___T,"ZJDLRC___T as dealer_name,"
6,会社コード,company_code,ZJCUST__0COMP_CODE,"ZJCUST__0COMP_CODE as company_code,"
7,会社,company_name,ZJCUST__0COMP_CODE___T,"ZJCUST__0COMP_CODE___T as company_name,"
8,CCBJIチャネルLv.1コード,ccbji_channel_lv.1_code,ZJBJICLV1,"ZJBJICLV1 as ccbji_channel_lv.1_code,"
9,CCBJIチャネルLv.1,ccbji_channel_lv.1_name,ZJBJICLV1___T,"ZJBJICLV1___T as ccbji_channel_lv.1_name,"


In [2]:
cols = ['account_code', 'account_name', 'customer_code', 'customer_name', 'jan_code', 'product_name',
        'manufacturer_code', 'manufacturer_name', 'scategory_l_code', 'scategory_l_name',
        'scategory_m_code', 'scategory_m_name', 'scategory_s_code', 'scategory_s_name','scategory_ss_code',
        'scategory_ss_name', 'norm', 'norm_scale', 'duration', 'sales_amount', 'sales_price', 'sales_revenue',
        'customer_engagement']

In [3]:
df = read_csv('C:/Users/tsubasa.ohmura/Downloads/Output/Output.csv', names=cols)
df = df[1:]

c:\users\tsubasa.ohmura\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,2,3,4,6,8,10,12,14,15,16,17,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def pos_trans(dataframe):
    df = dataframe
    #df = df.astype(str)
    df = df.astype({'sales_amount':int, 'sales_price':int, 'sales_revenue':int, 'customer_engagement':int})
    df['duration'] = to_datetime(df['duration'])
    return df

In [65]:
test = df[:10000].append(df[95900000:])

In [8]:
pos_trans(df[1:])

ValueError: invalid literal for int() with base 10: '117.0000'

In [9]:
df[1:]

,account_code,account_name,customer_code,customer_name,jan_code,product_name,manufacturer_code,manufacturer_name,scategory_l_code,scategory_l_name,scategory_m_code,scategory_m_name,scategory_s_code,scategory_s_name,scategory_ss_code,scategory_ss_name,norm,norm_scale,duration,sales_amount,sales_price,sales_revenue,customer_engagement
1,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999,分析対象外,999,対象外,999,対象外,NaN,NaN,0,NaN,2021-01-01,117.0000,116950.0000,0.0000,0
2,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999,分析対象外,999,対象外,999,対象外,NaN,NaN,0,NaN,2021-01-02,25.0000,24800.0000,0.0000,0
3,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999,分析対象外,999,対象外,999,対象外,NaN,NaN,0,NaN,2021-01-03,6.0000,6000.0000,0.0000,0
4,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999,分析対象外,999,対象外,999,対象外,NaN,NaN,0,NaN,2021-01-04,2.0000,2000.0000,0.0000,0
5,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999,分析対象外,999,対象外,999,対象外,NaN,NaN,0,NaN,2021-01-05,2.0000,2000.0000,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96906914,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035,大塚,7,スポーツ機能,1,スポーツ機能,1,スポーツ,NaN,NaN,0,NaN,2021-01-09,2,356,0,0
96906915,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035,大塚,7,スポーツ機能,1,スポーツ機能,1,スポーツ,NaN,NaN,0,NaN,2021-01-10,1,178,0,0
96906916,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035,大塚,7,スポーツ機能,1,スポーツ機能,1,スポーツ,NaN,NaN,0,NaN,2021-01-14,1,178,0,0
96906917,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035,大塚,7,スポーツ機能,1,スポーツ機能,1,スポーツ,NaN,NaN,0,NaN,2021-01-24,2,356,0,0


In [67]:
test = test.reset_index(drop=True)

In [68]:
test = pos_trans(test)

In [69]:
test

,account_code,account_name,customer_code,customer_name,jan_code,product_name,manufacturer_code,manufacturer_code,scategory_l_code,scategory_l_name,scategory_m_code,scategory_m_name,scategory_s_code,scategory_s_name,scategory_ss_code,scategory_ss_name,norm,norm_scale,duration,sales_amount,sales_price,sales_revenue,customer_engagement
0,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999.0,分析対象外,999.0,対象外,999.0,対象外,NaN,NaN,0,NaN,2021-01-01,117,116950,0,0
1,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999.0,分析対象外,999.0,対象外,999.0,対象外,NaN,NaN,0,NaN,2021-01-02,25,24800,0,0
2,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999.0,分析対象外,999.0,対象外,999.0,対象外,NaN,NaN,0,NaN,2021-01-03,6,6000,0,0
3,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999.0,分析対象外,999.0,対象外,999.0,対象外,NaN,NaN,0,NaN,2021-01-04,2,2000,0,0
4,11040,イオン北海道,NaN,NaN,2002301244105,対象外,NaN,NaN,999.0,分析対象外,999.0,対象外,999.0,対象外,NaN,NaN,0,NaN,2021-01-05,2,2000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016913,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035.0,大塚,7.0,スポーツ機能,1.0,スポーツ機能,1.0,スポーツ,NaN,NaN,0,NaN,2021-01-09,2,356,0,0
1016914,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035.0,大塚,7.0,スポーツ機能,1.0,スポーツ機能,1.0,スポーツ,NaN,NaN,0,NaN,2021-01-10,1,178,0,0
1016915,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035.0,大塚,7.0,スポーツ機能,1.0,スポーツ機能,1.0,スポーツ,NaN,NaN,0,NaN,2021-01-14,1,178,0,0
1016916,900087TK,大学生協（東海）,519501,夢プなんでも,4987035332510,ﾎﾟｶﾘｽｴﾂﾄ PB900ML,4987035.0,大塚,7.0,スポーツ機能,1.0,スポーツ機能,1.0,スポーツ,NaN,NaN,0,NaN,2021-01-24,2,356,0,0
